# 重构

- 一个影片出租的例子, 计算每位顾客消费金额, 并打印详单
- 顾客: 租了那些影片, 租期多长, 常客积分(根据片种和是否为新片而不同)
- 影片: 分类(儿童,普通, 新片)

In [9]:
# 电影类
class Movie(object):
    CHILDREN = 2
    REGULAR = 0
    NEW_RELEASE = 1
    def __init__(self, title, priceCode):
        self._title = title
        self._priceCode = priceCode
    def getPriceCode(self):
        return self._priceCode
    def setPriceCode(self, arg):
        self._priceCode = arg
    def getTitle(self):
        return self._title
    PriceCode = property(getPriceCode, setPriceCode)
    del getPriceCode, setPriceCode
# 租赁行为类
class Rental(object):
    def __init__(self, movie, daysRented):
        self._movie = movie
        self._daysRented = daysRented
    def getDaysRented(self):
        return self._daysRented
    def getMovie(self):
        return self._movie
# 顾客类
class Customer(object):
    def __init__(self, name):
        self._name = name
        self._rentals = []
    def addRental(self, arg):
        self._rentals.append(arg)
    def getName(self):
        return self._name
    def statement(self):
        totalAmount = 0
        frequentRenterPoints = 0
        result = "Rental Record for " + self.getName() + "\n"
        # determine amoiunts for each line
        for rental in self._rentals:
            thisAmount = 0
            if rental.getMovie().getPriceCode() == 0:
                thisAmount += 2
                if rental.getDaysRented() > 2:
                    thisAmount += (rental.getDaysRented() - 2) * 1.5
            elif rental.getMovie().getPriceCode() == 1:
                thisAmount += rental.getDaysRented() * 3
            else:
                thisAmount += 1.5
                if rental.getDaysRented() > 3:
                    thisAmount += (rental.getDaysRented() - 3) * 1.5
            # add frequent renter points
            frequentRenterPoints += 1
            # add bonus for a two day new release rental
            if (rental.getMovie().getPriceCode() == 1) and (rental.getDaysRented() > 1):
                frequentRenterPoints += 1
            # show figures for this rental
            result += "\t" + rental.getMovie().getTitle() + "\t" + str(thisAmount) + "\n"
            totalAmount += thisAmount
        # add footer lines
        result += "Amount owed is " + str(totalAmount) + "\n"
        result += "You earned " + str(frequentRenterPoints) + " frequent renter points"
        return result

### 需求调整

- 希望增加HTML输出详单
    - 无法复用 `statement()`, 重写一个`HTMLStatement()`, 大量复制`statement()`行为
    - 计费标准调整: 同时修改`statement()`和`HTMLStatement()`, 如果程序可能需要修改, 复制粘贴行为存在潜在威胁
    - 改变影片分类, 影响消费和常客积分计算方式, 

> 如果你为程序添加一个特性, 而代码结构无法方便的达成目的, 先重构那个程序, 是特性的添加比较容易, 再添加特性

### 重构的第一步

- 建立一套可靠的测试环境, 避免引入BUG
    - 先假设一些顾客, 各自租几个不同影片, 然后产生报表,并和参考报表比较
    - 测试必须要有自我检查能力
    
### 分解并重组`statement()`

- 代码越小, 功能越容易管理
- 把长函数切成小块代码, 并移至更合适的类, 以降低重复量


1. **Extract Method**
    - `if..elif..else` 语句
    - 首先找到函数内部的局部变量和参数, `rental`和`thisAmount`, 前者未被修改, 后者会被修改, __任何不会被修改的变量都可以被当成参数传入新的函数__, 会被修改的变量就需要格外小心.如果只有一个变量会被修改, 可以当成返回值
    - `thisAmount` 在每次循环起始处设置为0, 在if之前不会改变, 可以直接把新函数的返回值赋值给它

In [13]:
# 重构后的

class Customer(object):
    # 省略
    def statement(self):
        totalAmount = 0
        frequentRenterPoints = 0
        result = "Rental Record for " + self.getName() + "\n"
        # determine amoiunts for each line
        for rental in self._rentals:
            thisAmount = 0
            thisAmount = amountFor(rental)
            # add frequent renter points
            frequentRenterPoints += 1
            # add bonus for a two day new release rental
            if (rental.getMovie().getPriceCode() == 1) and (rental.getDaysRented() > 1):
                frequentRenterPoints += 1
            # show figures for this rental
            result += "\t" + rental.getMovie().getTitle() + "\t" + str(thisAmount) + "\n"
            totalAmount += thisAmount
        # add footer lines
        result += "Amount owed is " + str(totalAmount) + "\n"
        result += "You earned " + str(frequentRenterPoints) + " frequent renter points"
        return result
    # 注意 返回值类型
    def amountFor(self, rental):
        thisAmount = 0
        if rental.getMovie().getPriceCode() == 0:
            thisAmount += 2
            if rental.getDaysRented() > 2:
                thisAmount += (rental.getDaysRented() - 2) * 1.5
        elif rental.getMovie().getPriceCode() == 1:
            thisAmount += rental.getDaysRented() * 3
        else:
            thisAmount += 1.5
            if rental.getDaysRented() > 3:
                thisAmount += (rental.getDaysRented() - 3) * 1.5
        return thisAmount
# TODO: 看一下python extract method工具$chufengze$2018年11月15日$

- 更进一步, 修改变量名称
    - 唯有写出人类容易理解的代码,才是优秀的程序员
- 代码应该表现自己的目的

In [14]:
# 修改变量名
def amountFor(rental):
    result = 0
    if rental.getMovie().getPriceCode() == 0:
        result += 2
        if rental.getDaysRented() > 2:
            result += (rental.getDaysRented() - 2) * 1.5
    elif rental.getMovie().getPriceCode() == 1:
        result += rental.getDaysRented() * 3
    else:
        result += 1.5
        if rental.getDaysRented() > 3:
            result += (rental.getDaysRented() - 3) * 1.5
    return result

- `amountFor()`使用了`Rental`类的信息, 并没有使用`Customer`的信息
- 函数应该放在它所使用的数据所属对象内,所以`amountFor()`应该移动到`Rental`类内


2. **Move Method**

In [ ]:
class Rental(object):
    def getCharge(self):
        result = 0
        if self.getMovie().getPriceCode() == 0:
            result += 2
            if self.getDaysRented() > 2:
                result += (self.getDaysRented() - 2) * 1.5
        elif self.getMovie().getPriceCode() == 1:
            result += self.getDaysRented() * 3
        else:
            result += 1.5
            if self.getDaysRented() > 3:
                result += (self.getDaysRented() - 3) * 1.5
        return result
class Customer(object):
    def amountFor(self, rental):
        return rental.getCharge()
    def statement(self):
        thisAmount = rental.getCharge()